# Notebook for Data cleaning and Imputation

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer # noqa
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from utils.data_processing_utils import london_cleaner, hamburg_cleaner, stockholm_cleaner, boston_cleaner, \
                                        chicago_cleaner, houston_cleaner, save_df, valid_df, get_indices_of_rows_missing_data, preprocess_impute_fill, valid_splits_time
# from utils.data_processing_utils import *

## Constants 

### Paths

In [2]:
RAW_DATA_PATH = "Marathons_Data/Raw"
CLN_DATA_PATH = "Marathons_Data/Clean"
IMP_DATA_PATH = "Marathons_Data/Impute"
# London
LDN: str = "London"
LDN_RAW_PATH: str = f"{RAW_DATA_PATH}/{LDN}"
LDN_CLN_PATH: str = f"{CLN_DATA_PATH}/{LDN}"
LDN_IMP_PATH: str = f"{IMP_DATA_PATH}/{LDN}"
# Hamburg
HAM: str = "Hamburg"
HAM_RAW_PATH: str = f"{RAW_DATA_PATH}/{HAM}"
HAM_CLN_PATH: str = f"{CLN_DATA_PATH}/{HAM}"
HAM_IMP_PATH: str = f"{IMP_DATA_PATH}/{HAM}"
# Houston
HOU: str = "Houston"
HOU_RAW_PATH: str = f"{RAW_DATA_PATH}/{HOU}"
HOU_CLN_PATH: str = f"{CLN_DATA_PATH}/{HOU}"
HOU_IMP_PATH: str = f"{IMP_DATA_PATH}/{HOU}"
# Stockholm
STO: str = "Stockholm"
STO_RAW_PATH: str = f"{RAW_DATA_PATH}/{STO}"
STO_CLN_PATH: str = f"{CLN_DATA_PATH}/{STO}"
STO_IMP_PATH: str = f"{IMP_DATA_PATH}/{STO}"
# Boston
BOS: str = "Boston"
BOS_RAW_PATH: str = f"{RAW_DATA_PATH}/{BOS}"
BOS_CLN_PATH: str = f"{CLN_DATA_PATH}/{BOS}"
BOS_IMP_PATH: str = f"{IMP_DATA_PATH}/{BOS}"
# Chicago
CHI: str = "Chicago"
CHI_RAW_PATH: str = f"{RAW_DATA_PATH}/{CHI}"
CHI_CLN_PATH: str = f"{CLN_DATA_PATH}/{CHI}"
CHI_IMP_PATH: str = f"{IMP_DATA_PATH}/{CHI}"

### Years & Splits

In [3]:
YEAR_13: str = "2013"
YEAR_14: str = "2014"
YEAR_15: str = "2015"
YEAR_16: str = "2016"
YEAR_17: str = "2017"
YEAR_18: str = "2018"
YEAR_19: str = "2019"
YEAR_21: str = "2021"
YEAR_22: str = "2022"
YEAR_23: str = "2023"
YEARS: list[str] = [YEAR_13, YEAR_14, YEAR_15, YEAR_16, YEAR_17, YEAR_18, YEAR_19, YEAR_21, YEAR_22, YEAR_23]
SPLITS_KEYS: list[str] = ["k_5", "k_10", "k_15", "k_20", "k_half", "k_25", "k_30", "k_35", "k_40", "k_finish"]

COLS_ORDER: list[str] = ["age_cat", "gender", "race_state", "last_split", 
                         'k_5_time', 'k_5_pace', 'k_5_speed', 'k_10_time', 'k_10_pace', 'k_10_speed',
                         'k_15_time', 'k_15_pace', 'k_15_speed', 'k_20_time', 'k_20_pace', 'k_20_speed',
                         'k_half_time', 'k_half_pace', 'k_half_speed', 'k_25_time', 'k_25_pace', 'k_25_speed', 
                         'k_30_time', 'k_30_pace', 'k_30_speed', 'k_35_time', 'k_35_pace', 'k_35_speed',
                         'k_40_time', 'k_40_pace', 'k_40_speed', 'k_finish_time', 'k_finish_pace', 'k_finish_speed']

SPLIT_NAME_DICT: dict = {'k_5_time': '5K', 'k_10_time': '10K', 'k_15_time': '15K', 
                         'k_20_time': '20K', 'k_25_time': '25K', 'k_half_time': 'HALF', 
                         'k_30_time': '30K', 'k_35_time': '35K', 'k_40_time': '40K', 'k_finish_time': 'Finish time'}

DTYPE_DICT: defaultdict = defaultdict(np.float64, age_cat="category", gender="category", race_state="category", last_split="category")

In [4]:
# Intialise MinMaxScaler.
mms = MinMaxScaler()
# Intialise Imputers
knn_imputer = KNNImputer()
rfr = RandomForestRegressor(n_estimators=5, max_depth=10, bootstrap=True, max_samples=0.5, n_jobs=2, random_state=17)
iter_imputer = IterativeImputer(estimator=rfr, max_iter=15, random_state=17)

## London

In [5]:
LDN_COLS_TO_DROP = ["idp", "half", "finish", "run_no"] 

### 2014

In [6]:
# Raw file path.
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_14}/{LDN}{YEAR_14}_full.csv"
# Clean file path.
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_14}/{LDN}{YEAR_14}_clean.csv"
# Imputed files paths.
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_14}/{LDN}{YEAR_14}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_14}/{LDN}{YEAR_14}_iter_impute.csv"

#### Cleaning

In [6]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 41677 || New rows count: 36289 || Dropped Rows: 5388
** Dropping rows with splits that only contain time: Finished: 328 || Started: 27
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 35934 || New rows count: 35928 || Dropped rows based on   age_cat  : 6
Original rows count: 35928 || New rows count: 35928 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2014/London2014_clean.csv`


#### Imputation

In [7]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [8]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

In [9]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [10]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [11]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter 

### 2015

In [12]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_15}/{LDN}{YEAR_15}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_15}/{LDN}{YEAR_15}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_15}/{LDN}{YEAR_15}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_15}/{LDN}{YEAR_15}_iter_impute.csv"

#### Cleaning

In [9]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 43741 || New rows count: 37879 || Dropped Rows: 5862
** Dropping rows with splits that only contain time: Finished: 846 || Started: 17
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 37016 || New rows count: 36990 || Dropped rows based on   age_cat  : 26
Original rows count: 36990 || New rows count: 36990 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2015/London2015_clean.csv`


#### Imputation

In [13]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [14]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

In [15]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [16]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [17]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter 

### 2016

In [18]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_16}/{LDN}{YEAR_16}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_16}/{LDN}{YEAR_16}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_16}/{LDN}{YEAR_16}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_16}/{LDN}{YEAR_16}_iter_impute.csv"

#### Cleaning

In [12]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 45202 || New rows count: 39217 || Dropped Rows: 5985
** Dropping rows with splits that only contain time: Finished: 448 || Started: 9
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 38760 || New rows count: 38760 || Dropped rows based on   age_cat  : 0
Original rows count: 38760 || New rows count: 38760 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2016/London2016_clean.csv`


#### Imputation

In [19]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [20]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

In [21]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [22]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [23]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter 

### 2017

In [24]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_17}/{LDN}{YEAR_17}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_17}/{LDN}{YEAR_17}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_17}/{LDN}{YEAR_17}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_17}/{LDN}{YEAR_17}_iter_impute.csv"

#### Cleaning

In [15]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 45155 || New rows count: 39692 || Dropped Rows: 5463
** Dropping rows with splits that only contain time: Finished: 379 || Started: 16
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 39297 || New rows count: 39296 || Dropped rows based on   age_cat  : 1
Original rows count: 39296 || New rows count: 39296 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2017/London2017_clean.csv`


#### Imputation

In [25]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [26]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

In [27]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [28]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [29]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter

### 2018

In [30]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_18}/{LDN}{YEAR_18}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_18}/{LDN}{YEAR_18}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_18}/{LDN}{YEAR_18}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_18}/{LDN}{YEAR_18}_iter_impute.csv"

#### Cleaning

In [18]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 47667 || New rows count: 40773 || Dropped Rows: 6894
** Dropping rows with splits that only contain time: Finished: 338 || Started: 7
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 40428 || New rows count: 40416 || Dropped rows based on   age_cat  : 12
Original rows count: 40416 || New rows count: 40416 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2018/London2018_clean.csv`


#### Imputation

In [31]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [32]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

In [33]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [34]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [35]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter

### 2019

In [15]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_19}/{LDN}{YEAR_19}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_19}/{LDN}{YEAR_19}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_19}/{LDN}{YEAR_19}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_19}/{LDN}{YEAR_19}_iter_impute.csv"

#### Cleaning

In [21]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 49318 || New rows count: 42737 || Dropped Rows: 6581
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 42737 || New rows count: 42737 || Dropped rows based on   age_cat  : 0
Original rows count: 42737 || New rows count: 42737 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2019/London2019_clean.csv`


#### Imputation

In [16]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [17]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_10_time > k_15_time
Index([12700], dtype='int64')
Invalid split time: k_15_time > k_20_time
Index([19882], dtype='int64')
Invalid split time: k_20_time > k_half_time
Index([1121, 32585, 37157], dtype='int64')
Invalid split time: k_half_time > k_25_time
Index([28912, 40935], dtype='int64')
Invalid split time: k_30_time > k_35_time
Index([2575], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([6714, 41072, 41526, 42712], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([40799], dtype='int64')

** Dropping invalid splits, Count: 13


In [10]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_20_time > k_half_time
Index([1121], dtype='int64')
Invalid split time: k_30_time > k_35_time
Index([22476], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([6714, 41072, 41526, 42712, 42729], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([4804, 40799], dtype='int64')

** Dropping invalid splits, Count: 9


/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [11]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [12]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter

### 2021

In [36]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_21}/{LDN}{YEAR_21}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_21}/{LDN}{YEAR_21}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_21}/{LDN}{YEAR_21}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_21}/{LDN}{YEAR_21}_iter_impute.csv"

#### Cleaning

In [24]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 41594 || New rows count: 36129 || Dropped Rows: 5465
** Dropping rows with splits that only contain time: Finished: 2 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 36127 || New rows count: 36124 || Dropped rows based on   age_cat  : 3
Original rows count: 36124 || New rows count: 36124 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2021/London2021_clean.csv`


#### Imputation

In [37]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [38]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_15_time > k_20_time
Index([27209], dtype='int64')
Invalid split time: k_20_time > k_half_time
Index([18720], dtype='int64')
Invalid split time: k_half_time > k_25_time
Index([23994], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([34188], dtype='int64')

** Dropping invalid splits, Count: 4


In [39]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_20_time > k_half_time
Index([18720, 20164], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([34188], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([12428], dtype='int64')

** Dropping invalid splits, Count: 4


/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [40]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [41]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter

### 2022

In [42]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_22}/{LDN}{YEAR_22}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_22}/{LDN}{YEAR_22}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_22}/{LDN}{YEAR_22}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_22}/{LDN}{YEAR_22}_iter_impute.csv"

#### Cleaning

In [27]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 46993 || New rows count: 40812 || Dropped Rows: 6181
** Dropping rows with splits that only contain time: Finished: 1 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 40811 || New rows count: 40810 || Dropped rows based on   age_cat  : 1
Original rows count: 40810 || New rows count: 40810 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2022/London2022_clean.csv`


#### Imputation

In [43]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [44]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_10_time > k_15_time
Index([1490, 5053, 23642], dtype='int64')
Invalid split time: k_15_time > k_20_time
Index([30330], dtype='int64')
Invalid split time: k_20_time > k_half_time
Index([9206], dtype='int64')
Invalid split time: k_25_time > k_30_time
Index([36251], dtype='int64')
Invalid split time: k_30_time > k_35_time
Index([40288], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([22790], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([18729, 25420, 26885, 36585, 40200], dtype='int64')

** Dropping invalid splits, Count: 13


In [45]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_35_time > k_40_time
Index([22790], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([18729, 25420, 26885, 36585, 40200], dtype='int64')

** Dropping invalid splits, Count: 6


/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [46]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [47]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter

### 2023

In [48]:
ldn_file_raw_path = f"{LDN_RAW_PATH}/{LDN}{YEAR_23}/{LDN}{YEAR_23}_full.csv"
ldn_file_cln_path = f"{LDN_CLN_PATH}/{LDN}{YEAR_23}/{LDN}{YEAR_23}_clean.csv"
ldn_file_knn_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_23}/{LDN}{YEAR_23}_knn_impute.csv"
ldn_file_iter_imp_path = f"{LDN_IMP_PATH}/{LDN}{YEAR_23}/{LDN}{YEAR_23}_iter_impute.csv"

#### Cleaning

In [30]:
df_ldn = pd.read_csv(ldn_file_raw_path)
# Cleaning the DataFrame.
df_ldn = london_cleaner(df_ldn, SPLITS_KEYS, LDN_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ldn):
    save_df(df_ldn, ldn_file_cln_path)
    print(f"** File has been saved in: `{ldn_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 53077 || New rows count: 49083 || Dropped Rows: 3994
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 49083 || New rows count: 49083 || Dropped rows based on   age_cat  : 0
Original rows count: 49083 || New rows count: 49083 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80-84', '80+', '85+' by '70+'
** File has been saved in: `Marathons_Data/Clean/London/London2023/London2023_clean.csv`


#### Imputation

In [49]:
df_ldn = pd.read_csv(ldn_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ldn, SPLITS_KEYS)

In [50]:
df_knn = preprocess_impute_fill(df_ldn, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_10_time > k_15_time
Index([3951, 6047, 24950, 30864], dtype='int64')
Invalid split time: k_15_time > k_20_time
Index([8828, 24406, 45505, 45506], dtype='int64')
Invalid split time: k_20_time > k_half_time
Index([13156, 13340, 14615, 18257, 18874, 31444, 37310, 48044], dtype='int64')
Invalid split time: k_25_time > k_30_time
Index([4857], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([38209], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([15807, 19197, 24305, 30726, 38348], dtype='int64')

** Dropping invalid splits, Count: 23


In [51]:
df_iter = preprocess_impute_fill(df_ldn, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_15_time > k_20_time
Index([8828, 45505, 45506], dtype='int64')
Invalid split time: k_20_time > k_half_time
Index([13156, 18257], dtype='int64')
Invalid split time: k_25_time > k_30_time
Index([2893, 4857], dtype='int64')
Invalid split time: k_35_time > k_40_time
Index([38209], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([15807, 19197, 24305, 30726, 33305], dtype='int64')

** Dropping invalid splits, Count: 13


/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [52]:
save_df(df_knn, ldn_file_knn_imp_path)
save_df(df_iter, ldn_file_iter_imp_path)

In [53]:
del ldn_file_raw_path, ldn_file_cln_path, ldn_file_knn_imp_path, ldn_file_iter_imp_path, df_ldn, df_knn, df_iter

## Hamburg

In [4]:
HAM_COLS_TO_DROP = ["idp", "finish", "run_no"] 

### 2013

In [64]:
# Raw file path.
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_13}/{HAM}{YEAR_13}_full.csv"
# Clean file path.
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_13}/{HAM}{YEAR_13}_clean.csv"
# Imputed file path.
ham_file_knn_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_13}/{HAM}{YEAR_13}_knn_impute.csv"
ham_file_iter_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_13}/{HAM}{YEAR_13}_iter_impute.csv"


#### Cleaning

In [6]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 15135 || New rows count: 11872 || Dropped Rows: 3263
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 11872 || New rows count: 11872 || Dropped rows based on   age_cat  : 0
Original rows count: 11872 || New rows count: 11872 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 29 || Started: 1
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2013/Hamburg2013_clean.csv`


#### Imputation

In [55]:
df_ham = pd.read_csv(ham_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ham, SPLITS_KEYS)

In [57]:
df_knn = preprocess_impute_fill(df_ham, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_35_time > k_40_time
Index([145, 187, 11212], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([246, 644, 1454, 2515, 7170, 8020, 8668, 9895, 10138, 10215, 11720], dtype='int64')

** Dropping invalid splits, Count: 14


In [62]:
df_iter = preprocess_impute_fill(df_ham, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_35_time > k_40_time
Index([145, 187, 11212], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([246, 644, 1454, 7170, 8020, 8668, 9895, 10138, 10215], dtype='int64')

** Dropping invalid splits, Count: 12


/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [65]:
save_df(df_knn, ham_file_knn_imp_path)
save_df(df_iter, ham_file_iter_imp_path)

In [66]:
del ham_file_raw_path, ham_file_cln_path, ham_file_knn_imp_path, ham_file_iter_imp_path, df_ham, df_knn, df_iter

### 2014

In [67]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_14}/{HAM}{YEAR_14}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_14}/{HAM}{YEAR_14}_clean.csv"
ham_file_knn_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_14}/{HAM}{YEAR_14}_knn_impute.csv"
ham_file_iter_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_14}/{HAM}{YEAR_14}_iter_impute.csv"

#### Cleaning

In [9]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 16695 || New rows count: 13296 || Dropped Rows: 3399
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 13296 || New rows count: 13295 || Dropped rows based on   age_cat  : 1
Original rows count: 13295 || New rows count: 13295 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 13 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2014/Hamburg2014_clean.csv`


#### Imputation

In [68]:
df_ham = pd.read_csv(ham_file_cln_path, dtype=DTYPE_DICT)
missing_indices = get_indices_of_rows_missing_data(df_ham, SPLITS_KEYS)

In [69]:
df_knn = preprocess_impute_fill(df_ham, missing_indices, knn_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_30_time > k_35_time
Index([3457], dtype='int64')
Invalid split time: k_40_time > k_finish_time
Index([1364, 2274, 2994, 6173, 7970, 8358, 9051], dtype='int64')

** Dropping invalid splits, Count: 8


In [70]:
df_iter = preprocess_impute_fill(df_ham, missing_indices, iter_imputer, mms, SPLITS_KEYS, drop_invalid_splits=True)

Invalid split time: k_40_time > k_finish_time
Index([1364, 2274, 2994, 6173, 7970, 8358], dtype='int64')

** Dropping invalid splits, Count: 6


/Users/sika/miniforge3/envs/ML_mac_23/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [83]:
save_df(df_knn, ham_file_knn_imp_path)
save_df(df_iter, ham_file_iter_imp_path)

In [84]:
del ham_file_raw_path, ham_file_cln_path, ham_file_knn_imp_path, ham_file_iter_imp_path, df_ham, df_knn, df_iter

### 2015

In [11]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_15}/{HAM}{YEAR_15}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_15}/{HAM}{YEAR_15}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_15}/{HAM}{YEAR_15}_imputed.csv"

#### Cleaning

In [12]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 19205 || New rows count: 15259 || Dropped Rows: 3946
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 15259 || New rows count: 15257 || Dropped rows based on   age_cat  : 2
Original rows count: 15257 || New rows count: 15257 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 17 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2015/Hamburg2015_clean.csv`


#### Imputation

In [13]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

### 2016

In [14]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_16}/{HAM}{YEAR_16}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_16}/{HAM}{YEAR_16}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_16}/{HAM}{YEAR_16}_imputed.csv"

#### Cleaning

In [15]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 16011 || New rows count: 12540 || Dropped Rows: 3471
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 12540 || New rows count: 12537 || Dropped rows based on   age_cat  : 3
Original rows count: 12537 || New rows count: 12537 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 5 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2016/Hamburg2016_clean.csv`


#### Imputation

In [16]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

### 2017

In [17]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_17}/{HAM}{YEAR_17}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_17}/{HAM}{YEAR_17}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_17}/{HAM}{YEAR_17}_imputed.csv"

#### Cleaning

In [18]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 15638 || New rows count: 12396 || Dropped Rows: 3242
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 12396 || New rows count: 12391 || Dropped rows based on   age_cat  : 5
Original rows count: 12391 || New rows count: 12391 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 15 || Started: 3
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2017/Hamburg2017_clean.csv`


#### Imputation

In [19]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

### 2018

In [21]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_18}/{HAM}{YEAR_18}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_18}/{HAM}{YEAR_18}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_18}/{HAM}{YEAR_18}_imputed.csv"

#### Cleaning

In [22]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 14010 || New rows count: 10670 || Dropped Rows: 3340
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 10670 || New rows count: 10668 || Dropped rows based on   age_cat  : 2
Original rows count: 10668 || New rows count: 10668 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2018/Hamburg2018_clean.csv`


#### Imputation

In [23]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

### 2019

In [24]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_19}/{HAM}{YEAR_19}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_19}/{HAM}{YEAR_19}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_19}/{HAM}{YEAR_19}_imputed.csv"

#### Cleaning

In [25]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 13498 || New rows count: 10468 || Dropped Rows: 3030
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 10468 || New rows count: 10453 || Dropped rows based on   age_cat  : 15
Original rows count: 10453 || New rows count: 10453 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2019/Hamburg2019_clean.csv`


#### Imputation

In [26]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

### 2022

In [27]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_22}/{HAM}{YEAR_22}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_22}/{HAM}{YEAR_22}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_22}/{HAM}{YEAR_22}_imputed.csv"

#### Cleaning

In [28]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 10416 || New rows count: 6888 || Dropped Rows: 3528
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 6888 || New rows count: 6840 || Dropped rows based on   age_cat  : 48
Original rows count: 6840 || New rows count: 6840 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2022/Hamburg2022_clean.csv`


#### Imputation

In [29]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

### 2023

In [31]:
ham_file_raw_path = f"{HAM_RAW_PATH}/{HAM}{YEAR_23}/{HAM}{YEAR_23}_full.csv"
ham_file_cln_path = f"{HAM_CLN_PATH}/{HAM}{YEAR_23}/{HAM}{YEAR_23}_clean.csv"
ham_file_imp_path = f"{HAM_IMP_PATH}/{HAM}{YEAR_23}/{HAM}{YEAR_23}_imputed.csv"

#### Cleaning

In [32]:
df_ham = pd.read_csv(ham_file_raw_path)
# Cleaning The DataFrame
df_ham = hamburg_cleaner(df_ham, SPLITS_KEYS, HAM_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_ham):
    save_df(df_ham, ham_file_cln_path)
    print(f"** File has been saved in: `{ham_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 11757 || New rows count: 9002 || Dropped Rows: 2755
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 9002 || New rows count: 8998 || Dropped rows based on   age_cat  : 4
Original rows count: 8998 || New rows count: 8998 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** File has been saved in: `Marathons_Data/Clean/Hamburg/Hamburg2023/Hamburg2023_clean.csv`


#### Imputation

In [33]:
del ham_file_raw_path, ham_file_cln_path, ham_file_imp_path, df_ham

## Houston
##### Pace and speed have been converted from min/mile and miles/h to sec/km and km/h respectively.

In [4]:
HOU_COLS_TO_DROP = ["idp", "finish", "run_no"] 

### 2018

In [5]:
hou_file_raw_path = f"{HOU_RAW_PATH}/{HOU}{YEAR_18}/{HOU}{YEAR_18}_full.csv"
hou_file_cln_path = f"{HOU_CLN_PATH}/{HOU}{YEAR_18}/{HOU}{YEAR_18}_clean.csv"
hou_file_imp_path = f"{HOU_IMP_PATH}/{HOU}{YEAR_18}/{HOU}{YEAR_18}_imputed.csv"

#### Cleaning

In [6]:
df_hou = pd.read_csv(hou_file_raw_path)
# Cleaning the DataFrame.
df_hou = houston_cleaner(df_hou, SPLITS_KEYS, HOU_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_hou):
    save_df(df_hou, hou_file_cln_path)
    print(f"** File has been saved in: `{hou_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 7547 || New rows count: 7526 || Dropped Rows: 21
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 7526 || New rows count: 7526 || Dropped rows based on   age_cat  : 0
Original rows count: 7526 || New rows count: 7526 || Dropped rows based on    gender  : 0
** Dropping rows with invalid age categories [12-15, 16-19, Elites]:
Original rows count: 7526 || New rows count: 7388 || Dropped rows: 138
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** Dropping rows with invalid race state ['Other', 'DQ - No Reason Was Given', 'DQ - SWITCH from HALF to MARA']:
Original rows count: 7388 || New rows count: 7374 || Dropped rows: 14
** Dropping rows with splits that only contain time: Finished: 0 || Started: 4
** File has been saved in: `Marathons_Data/Clean/Houston/Houston2018/Houston2018_clean.c

#### Imputation

In [7]:
del hou_file_raw_path, hou_file_cln_path, hou_file_imp_path, df_hou

### 2019

In [8]:
hou_file_raw_path = f"{HOU_RAW_PATH}/{HOU}{YEAR_19}/{HOU}{YEAR_19}_full.csv"
hou_file_cln_path = f"{HOU_CLN_PATH}/{HOU}{YEAR_19}/{HOU}{YEAR_19}_clean.csv"
hou_file_imp_path = f"{HOU_IMP_PATH}/{HOU}{YEAR_19}/{HOU}{YEAR_19}_imputed.csv"

#### Cleaning

In [9]:
df_hou = pd.read_csv(hou_file_raw_path)
# Cleaning the DataFrame.
df_hou = houston_cleaner(df_hou, SPLITS_KEYS, HOU_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_hou):
    save_df(df_hou, hou_file_cln_path)
    print(f"** File has been saved in: `{hou_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 7159 || New rows count: 7145 || Dropped Rows: 14
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 7145 || New rows count: 7145 || Dropped rows based on   age_cat  : 0
Original rows count: 7145 || New rows count: 7145 || Dropped rows based on    gender  : 0
** Dropping rows with invalid age categories [12-15, 16-19, Elites]:
Original rows count: 7145 || New rows count: 7015 || Dropped rows: 130
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** Dropping rows with invalid race state ['Other', 'DQ - No Reason Was Given', 'DQ - SWITCH from HALF to MARA']:
Original rows count: 7015 || New rows count: 7003 || Dropped rows: 12
** Dropping rows with splits that only contain time: Finished: 0 || Started: 4
** File has been saved in: `Marathons_Data/Clean/Houston/Houston2019/Houston2019_clean.c

#### Imputation

In [10]:
del hou_file_raw_path, hou_file_cln_path, hou_file_imp_path, df_hou

## Stockholm

In [4]:
STO_COLS_TO_DROP = ["idp", "finish", "run_no"] 

### 2021

In [5]:
sto_file_raw_path = f"{STO_RAW_PATH}/{STO}{YEAR_21}/{STO}{YEAR_21}_full.csv"
sto_file_cln_path = f"{STO_CLN_PATH}/{STO}{YEAR_21}/{STO}{YEAR_21}_clean.csv"
sto_file_imp_path = f"{STO_IMP_PATH}/{STO}{YEAR_21}/{STO}{YEAR_21}_imputed.csv"

#### Cleaning

In [6]:
df_sto = pd.read_csv(sto_file_raw_path)
# Cleaning the DataFrame.
df_sto = stockholm_cleaner(df_sto, SPLITS_KEYS, STO_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER, year=2021)
# Check if the DataFrame is valid before saving it.
if valid_df(df_sto):
    save_df(df_sto, sto_file_cln_path)
    print(f"** File has been saved in: `{sto_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 12179 || New rows count: 7177 || Dropped Rows: 5002
** Dropping rows with null values in `yob` and `gender` columns:
Original rows count: 7177 || New rows count: 7126 || Dropped rows based on     yob    : 51
Original rows count: 7126 || New rows count: 7126 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping non-adult runners (age < 18):
Original rows count: 7126 || New rows count: 7125 || Dropped rows: 1
** column name `yob` changed to `age_cat`.
** File has been saved in: `Marathons_Data/Clean/Stockholm/Stockholm2021/Stockholm2021_clean.csv`


#### Imputation

In [7]:
del sto_file_raw_path, sto_file_cln_path, sto_file_imp_path, df_sto

### 2022

In [8]:
sto_file_raw_path = f"{STO_RAW_PATH}/{STO}{YEAR_22}/{STO}{YEAR_22}_full.csv"
sto_file_cln_path = f"{STO_CLN_PATH}/{STO}{YEAR_22}/{STO}{YEAR_22}_clean.csv"
sto_file_imp_path = f"{STO_IMP_PATH}/{STO}{YEAR_22}/{STO}{YEAR_22}_imputed.csv"

#### Cleaning

In [9]:
df_sto = pd.read_csv(sto_file_raw_path)
# Cleaning the DataFrame.
df_sto = stockholm_cleaner(df_sto, SPLITS_KEYS, STO_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER, year=2022)
# Check if the DataFrame is valid before saving it.
if valid_df(df_sto):
    save_df(df_sto, sto_file_cln_path)
    print(f"** File has been saved in: `{sto_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 13593 || New rows count: 10161 || Dropped Rows: 3432
** Dropping rows with null values in `yob` and `gender` columns:
Original rows count: 10161 || New rows count: 10057 || Dropped rows based on     yob    : 104
Original rows count: 10057 || New rows count: 10057 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping non-adult runners (age < 18):
Original rows count: 10057 || New rows count: 10057 || Dropped rows: 0
** column name `yob` changed to `age_cat`.
** File has been saved in: `Marathons_Data/Clean/Stockholm/Stockholm2022/Stockholm2022_clean.csv`


#### Imputation

In [10]:
del sto_file_raw_path, sto_file_cln_path, sto_file_imp_path, df_sto

## Boston
##### Pace and speed have been converted from min/mile and miles/h to sec/km and km/h respectively.

In [6]:
BOS_COLS_TO_DROP = ["idp", "finish", "run_no"] 

### 2014

In [7]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_14}/{BOS}{YEAR_14}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_14}/{BOS}{YEAR_14}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_14}/{BOS}{YEAR_14}_imputed.csv"

#### Cleaning

In [8]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 35671 || New rows count: 32447 || Dropped Rows: 3224
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 32123 || New rows count: 32123 || Dropped rows based on   age_cat  : 0
Original rows count: 32123 || New rows count: 32123 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2014/Boston2014_clean.csv`


#### Imputation

In [9]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2015

In [10]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_15}/{BOS}{YEAR_15}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_15}/{BOS}{YEAR_15}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_15}/{BOS}{YEAR_15}_imputed.csv"

#### Cleaning

In [11]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 30252 || New rows count: 27159 || Dropped Rows: 3093
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 26986 || New rows count: 26986 || Dropped rows based on   age_cat  : 0
Original rows count: 26986 || New rows count: 26986 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2015/Boston2015_clean.csv`


#### Imputation

In [12]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2016

In [13]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_16}/{BOS}{YEAR_16}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_16}/{BOS}{YEAR_16}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_16}/{BOS}{YEAR_16}_imputed.csv"

#### Cleaning

In [14]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 30743 || New rows count: 27487 || Dropped Rows: 3256
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 27421 || New rows count: 27421 || Dropped rows based on   age_cat  : 0
Original rows count: 27421 || New rows count: 27421 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2016/Boston2016_clean.csv`


#### Imputation

In [15]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2017

In [16]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_17}/{BOS}{YEAR_17}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_17}/{BOS}{YEAR_17}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_17}/{BOS}{YEAR_17}_imputed.csv"

#### Cleaning

In [17]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 30074 || New rows count: 27220 || Dropped Rows: 2854
** Dropping rows with splits that only contain time: Finished: 0 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 27189 || New rows count: 27189 || Dropped rows based on   age_cat  : 0
Original rows count: 27189 || New rows count: 27189 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2017/Boston2017_clean.csv`


#### Imputation

In [18]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2018

In [19]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_18}/{BOS}{YEAR_18}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_18}/{BOS}{YEAR_18}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_18}/{BOS}{YEAR_18}_imputed.csv"

#### Cleaning

In [20]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 29978 || New rows count: 26919 || Dropped Rows: 3059
** Dropping rows with splits that only contain time: Finished: 9 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 26910 || New rows count: 26910 || Dropped rows based on   age_cat  : 0
Original rows count: 26910 || New rows count: 26910 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2018/Boston2018_clean.csv`


#### Imputation

In [21]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2019

In [22]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_19}/{BOS}{YEAR_19}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_19}/{BOS}{YEAR_19}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_19}/{BOS}{YEAR_19}_imputed.csv"

#### Cleaning

In [23]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 30234 || New rows count: 27337 || Dropped Rows: 2897
** Dropping rows with splits that only contain time: Finished: 37 || Started: 1
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 27299 || New rows count: 27299 || Dropped rows based on   age_cat  : 0
Original rows count: 27299 || New rows count: 27299 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2019/Boston2019_clean.csv`


#### Imputation

In [24]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2021

In [25]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_21}/{BOS}{YEAR_21}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_21}/{BOS}{YEAR_21}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_21}/{BOS}{YEAR_21}_imputed.csv"

#### Cleaning

In [26]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 18074 || New rows count: 15645 || Dropped Rows: 2429
** Dropping rows with splits that only contain time: Finished: 1 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 15644 || New rows count: 15644 || Dropped rows based on   age_cat  : 0
Original rows count: 15644 || New rows count: 15644 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2021/Boston2021_clean.csv`


#### Imputation

In [27]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2022

In [28]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_22}/{BOS}{YEAR_22}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_22}/{BOS}{YEAR_22}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_22}/{BOS}{YEAR_22}_imputed.csv"

#### Cleaning

In [29]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 28500 || New rows count: 25217 || Dropped Rows: 3283
** Dropping rows with splits that only contain time: Finished: 4 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 25213 || New rows count: 25213 || Dropped rows based on   age_cat  : 0
Original rows count: 25213 || New rows count: 25213 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2022/Boston2022_clean.csv`


#### Imputation

In [30]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

### 2023

In [31]:
bos_file_raw_path = f"{BOS_RAW_PATH}/{BOS}{YEAR_23}/{BOS}{YEAR_23}_full.csv"
bos_file_cln_path = f"{BOS_CLN_PATH}/{BOS}{YEAR_23}/{BOS}{YEAR_23}_clean.csv"
bos_file_imp_path = f"{BOS_IMP_PATH}/{BOS}{YEAR_23}/{BOS}{YEAR_23}_imputed.csv"

#### Cleaning

In [32]:
df_bos = pd.read_csv(bos_file_raw_path, low_memory=False)
# # Cleaning the DataFrame.
df_bos = boston_cleaner(df_bos, SPLITS_KEYS, BOS_COLS_TO_DROP, COLS_ORDER)
# # Check if the DataFrame is valid before saving it.
if valid_df(df_bos):
    save_df(df_bos, bos_file_cln_path)
    print(f"** File has been saved in: `{bos_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 30105 || New rows count: 27058 || Dropped Rows: 3047
** Dropping rows with splits that only contain time: Finished: 3 || Started: 0
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 27055 || New rows count: 27055 || Dropped rows based on   age_cat  : 0
Original rows count: 27055 || New rows count: 27055 || Dropped rows based on    gender  : 0
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Boston/Boston2023/Boston2023_clean.csv`


#### Imputation

In [33]:
del bos_file_raw_path, bos_file_cln_path, bos_file_imp_path, df_bos

## Chicago

In [4]:
CHI_COLS_TO_DROP = ["idp", "finish", "run_no"] 

### 2014

In [5]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_14}/{CHI}{YEAR_14}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_14}/{CHI}{YEAR_14}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_14}/{CHI}{YEAR_14}_imputed.csv"

#### Cleaning

In [6]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 50216 || New rows count: 41715 || Dropped Rows: 8501
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 41715 || New rows count: 41715 || Dropped rows based on   age_cat  : 0
Original rows count: 41715 || New rows count: 41715 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 27 || Started: 0
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 41688 || New rows count: 41323 || Dropped rows: 365
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2014/Chicago2014_clean.csv`


#### Imputation

In [7]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2015

In [8]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_15}/{CHI}{YEAR_15}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_15}/{CHI}{YEAR_15}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_15}/{CHI}{YEAR_15}_imputed.csv"

#### Cleaning

In [9]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 46032 || New rows count: 39219 || Dropped Rows: 6813
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 39219 || New rows count: 39219 || Dropped rows based on   age_cat  : 0
Original rows count: 39219 || New rows count: 39219 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 18 || Started: 1
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 39200 || New rows count: 38868 || Dropped rows: 332
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2015/Chicago2015_clean.csv`


#### Imputation

In [10]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2016

In [11]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_16}/{CHI}{YEAR_16}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_16}/{CHI}{YEAR_16}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_16}/{CHI}{YEAR_16}_imputed.csv"

#### Cleaning

In [12]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 49067 || New rows count: 41469 || Dropped Rows: 7598
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 41469 || New rows count: 41468 || Dropped rows based on   age_cat  : 1
Original rows count: 41468 || New rows count: 41468 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 22 || Started: 0
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 41446 || New rows count: 41141 || Dropped rows: 305
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2016/Chicago2016_clean.csv`


#### Imputation

In [13]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2017

In [14]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_17}/{CHI}{YEAR_17}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_17}/{CHI}{YEAR_17}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_17}/{CHI}{YEAR_17}_imputed.csv"

#### Cleaning

In [15]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 54326 || New rows count: 45565 || Dropped Rows: 8761
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 45565 || New rows count: 45565 || Dropped rows based on   age_cat  : 0
Original rows count: 45565 || New rows count: 45565 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 34 || Started: 3
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 45528 || New rows count: 45289 || Dropped rows: 239
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2017/Chicago2017_clean.csv`


#### Imputation

In [16]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2018

In [17]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_18}/{CHI}{YEAR_18}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_18}/{CHI}{YEAR_18}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_18}/{CHI}{YEAR_18}_imputed.csv"

#### Cleaning

In [18]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 55621 || New rows count: 45380 || Dropped Rows: 10241
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 45380 || New rows count: 45380 || Dropped rows based on   age_cat  : 0
Original rows count: 45380 || New rows count: 45380 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 25 || Started: 1
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 45354 || New rows count: 45119 || Dropped rows: 235
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2018/Chicago2018_clean.csv`


#### Imputation

In [19]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2019

In [20]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_19}/{CHI}{YEAR_19}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_19}/{CHI}{YEAR_19}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_19}/{CHI}{YEAR_19}_imputed.csv"

#### Cleaning

In [21]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 55395 || New rows count: 46513 || Dropped Rows: 8882
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 46513 || New rows count: 46512 || Dropped rows based on   age_cat  : 1
Original rows count: 46512 || New rows count: 46512 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 4 || Started: 0
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 46508 || New rows count: 46265 || Dropped rows: 243
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2019/Chicago2019_clean.csv`


#### Imputation

In [22]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2021

In [23]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_21}/{CHI}{YEAR_21}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_21}/{CHI}{YEAR_21}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_21}/{CHI}{YEAR_21}_imputed.csv"

#### Cleaning

In [24]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 33502 || New rows count: 26864 || Dropped Rows: 6638
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 26864 || New rows count: 26864 || Dropped rows based on   age_cat  : 0
Original rows count: 26864 || New rows count: 26864 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 19 || Started: 0
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 26845 || New rows count: 26730 || Dropped rows: 115
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2021/Chicago2021_clean.csv`


#### Imputation

In [25]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi

### 2022

In [26]:
chi_file_raw_path = f"{CHI_RAW_PATH}/{CHI}{YEAR_22}/{CHI}{YEAR_22}_full.csv"
chi_file_cln_path = f"{CHI_CLN_PATH}/{CHI}{YEAR_22}/{CHI}{YEAR_22}_clean.csv"
chi_file_imp_path = f"{CHI_IMP_PATH}/{CHI}{YEAR_22}/{CHI}{YEAR_22}_imputed.csv"

#### Cleaning

In [27]:
df_chi = pd.read_csv(chi_file_raw_path)
# Cleaning the DataFrame.
df_chi = chicago_cleaner(df_chi, SPLITS_KEYS, CHI_COLS_TO_DROP, SPLIT_NAME_DICT, COLS_ORDER)
# Check if the DataFrame is valid before saving it.
if valid_df(df_chi):
    save_df(df_chi, chi_file_cln_path)
    print(f"** File has been saved in: `{chi_file_cln_path}`")

** Removing Runners That did not start:
Original rows count: 51087 || New rows count: 39939 || Dropped Rows: 11148
** Dropping rows with null values in `age_cat` and `gender` columns:
Original rows count: 39939 || New rows count: 39938 || Dropped rows based on   age_cat  : 1
Original rows count: 39938 || New rows count: 39938 || Dropped rows based on    gender  : 0
** Dropping rows with splits that only contain time: Finished: 7 || Started: 0
** Dropping rows with invalid age categories [W-15, M-15, 19 and under]:
Original rows count: 39931 || New rows count: 39790 || Dropped rows: 141
** Replacing these age categories '20-24', '25-29', '30-34', and '35-39' by '18-39'
** Replacing these age categories '70-74', '75-79', '80+' by '70+'
** File has been saved in: `Marathons_Data/Clean/Chicago/Chicago2022/Chicago2022_clean.csv`


#### Imputation

In [28]:
del chi_file_raw_path, chi_file_cln_path, chi_file_imp_path, df_chi